# Demo: Getting data from the Open Data Toronto portal

In [1]:
from shapely.geometry import shape
from ipyleaflet import *
import geopandas as gpd
import pandas as pd
import requests
import json

## Reading from the Open Data Portal via APIs

Our data can be accessed via API by anyone because it is built on CKAN, the world's leading open source data portal. [With the CKAN API you can retrieve information on the catalogue, datasets, and even the data itself](https://docs.ckan.org/en/2.8/api/).

> **Note on terminology**
> 
> In CKAN API terms:
>
>* **Package** refers to what we often call a "dataset"
>* **Resource** refers to what we often call a "file"
>
> Will use those terms going forward for consistency with API terms.

In [2]:
API_URL = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action"

### Getting metadata on the catalogue

#### Get list of dataset names

Easy to get a list of all package names in the catalogue.

In [3]:
package_list = requests.get(url=f"{API_URL}/package_list").json()

print(f"Total datasets: {len(package_list['result'])} \nExample:{package_list['result'][:5]}")

Total datasets: 331 
Example:['2013-street-needs-assessment-results', '2018-street-needs-assessment-results', '311-contact-centre-performance-metrics', '311-open311-api-calls-for-service-requests', '311-service-request-codes']


#### Searching the catalogue


Use `package_search` to retrieve a list of dictionaries, where each dictionary is the complete dataset metadata including the metadata of files (ie. resources) the dataset has. You can pass a query as demonstrated here (refer to the documentation for examples) to narrow the number of results.

In [4]:
query = "neighbourhood"
packages_query_search = requests.get(url=f"{API_URL}/package_search", params={"rows": 10000, "q": query}).json()

print(f"Datasets found for \"{query}\" : {packages_query_search['result']['count']}")

example = packages_query_search["result"]["results"][0]
print(json.dumps(example, indent=2))

Datasets found for "neighbourhood" : 25
{
  "license_title": "Open Government Licence \u2013 Toronto",
  "owner_unit": null,
  "relationships_as_object": [],
  "topics": "Locations and mapping",
  "owner_email": "opendata@toronto.ca",
  "excerpt": "Boundaries of Toronto Neighbourhoods.",
  "private": false,
  "owner_division": "Social Development, Finance & Administration",
  "num_tags": 9,
  "id": "4def3f65-2a65-4a4f-83c4-b2a4aed72d46",
  "metadata_created": "2019-07-23T17:47:38.364157",
  "refresh_rate": "As available",
  "title": "Neighbourhoods",
  "license_url": "https://open.toronto.ca/open-data-license/",
  "state": "active",
  "information_url": "http://www.toronto.ca/demographics/neighbourhoods.htm",
  "license_id": "open-government-licence-toronto",
  "type": "dataset",
  "resources": [
    {
      "cache_last_updated": null,
      "package_id": "4def3f65-2a65-4a4f-83c4-b2a4aed72d46",
      "datastore_active": true,
      "id": "a083c865-6d60-4d1d-b6c6-b0c8a85f9c15",
      "s

#### Retrieving metadata for the entire catalogue

You can use `package_search` without a query to match every dataset, just pass the `rows` argument to get everything - in this example it is set to 10000.

In [5]:
packages = requests.get(url=f"{API_URL}/package_search", params={"rows": 10000}).json()["result"]

print(f"{packages['count']} datasets found")

331 datasets found


### Getting specific datasets (packages)

Use `package_show` to get a specific dataset, just need to pass an `id` argument containing either the package `name` or `id` (as shown in the example dictionary above).

This will effectively return a single package in the same structure as the response from `package_search`. [Here's an example using Dinesafe](https://open.toronto.ca/dataset/dinesafe/):

In [6]:
def get_package(id_or_name):
    url = f"{API_URL}/package_show"
    params = { "id": id_or_name}
    return requests.get(url, params).json()

example_package = get_package("red-light-cameras")["result"]
print(json.dumps(example_package, indent=2))

{
  "license_title": "Open Government Licence \u2013 Toronto",
  "owner_unit": null,
  "relationships_as_object": [],
  "topics": "Locations and mapping,Public safety,Transportation",
  "owner_email": "opendata@toronto.ca",
  "excerpt": "This dataset identifies the intersections in Toronto where red light cameras are located.",
  "private": false,
  "owner_division": "Transportation Services",
  "num_tags": 11,
  "id": "9fcff3e1-3737-43cf-b410-05acd615e27b",
  "metadata_created": "2019-07-23T17:56:31.216945",
  "refresh_rate": "Annually",
  "title": "Red Light Cameras",
  "license_url": "https://open.toronto.ca/open-data-license/",
  "state": "active",
  "information_url": "https://www.toronto.ca/services-payments/streets-parking-transportation/traffic-management/",
  "license_id": "open-government-licence-toronto",
  "type": "dataset",
  "resources": [
    {
      "cache_last_updated": null,
      "package_id": "9fcff3e1-3737-43cf-b410-05acd615e27b",
      "datastore_active": true,
  

## Interactive Example

### Data overview

In [7]:
def get_package_tables(package_id_or_name):
    params = { "id": package_id_or_name}
    package = requests.get(f"{API_URL}/package_show", params).json()
    tables = []
    
    for idx, resource in enumerate(package["result"]["resources"]):
        if resource["datastore_active"]:
            url = f"{API_URL}/datastore_search"
            p = { "id": resource["id"] }
            total_record_count = requests.get(url, { "id": resource["id"] }).json()["result"]["total"]
            p["limit"] = total_record_count

            data = requests.get(url, p).json()
            df = pd.DataFrame(data["result"]["records"])
            tables.append(df)
            
    return tables

#### Fire Incidents

https://open.toronto.ca/dataset/fire-incidents/

In [8]:
incidents_tables = get_package_tables("fire-incidents")
print(f"Tables found: {len(incidents_tables)}")

Tables found: 1


In [9]:
incidents = incidents_tables[0]
incidents["TFS_Arrival_Time"]  = pd.to_datetime(incidents["TFS_Arrival_Time"] )
incidents["TFS_Alarm_Time"] = pd.to_datetime(incidents["TFS_Alarm_Time"])
# table["Response_Time"] = table["TFS_Alarm_Time"] - table["TFS_Arrival_Time"]

incidents.head()

,_id,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Exposures,Ext_agent_app_or_defer_time,...,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Alarm_Time,TFS_Arrival_Time,TFS_Firefighter_Casualties
0,403654,97 - Other - unclassified,None,None,0,0,10000.0,None,NaN,2014-12-23T16:46:00,...,None,None,None,None,None,None,1 - Fire extinguished prior to arrival,2014-12-23 16:39:25,2014-12-23 16:44:00,0
1,403655,81 - Engine Area,None,None,0,0,15000.0,None,NaN,2015-01-02T10:20:41,...,None,None,None,None,None,None,"7 - Fully involved (total structure, vehicle, ...",2015-01-02 10:14:25,2015-01-02 10:19:41,0
2,403656,81 - Engine Area,None,None,0,0,7000.0,None,NaN,2016-05-29T15:02:30,...,None,None,None,None,None,None,"7 - Fully involved (total structure, vehicle, ...",2016-05-29 14:57:31,2016-05-29 15:01:33,0
3,403657,79 - Other Outside Area,01 - Normal (no change),1 - No business interruption,0,0,0.0,0,NaN,2016-11-15T17:23:39,...,4 - Remote from fire – smoke did not reach alarm,8 - Not applicable - no smoke alarm or presenc...,7 - Not applicable: Occupant(s) first alerted ...,2 - Confined to part of room/area of origin,2 - Did not activate: remote from fire,9 - Undetermined,4 - Flames showing from small area (one storey...,2016-11-15 17:18:39,2016-11-15 17:22:12,0
4,403658,81 - Engine Area,None,None,0,0,6000.0,None,NaN,2013-03-09T15:03:00,...,None,None,None,None,None,None,3 - Fire with smoke showing only - including v...,2013-03-09 14:54:05,2013-03-09 14:59:45,0


#### Neighbourhoods

https://open.toronto.ca/dataset/neighbourhoods/

Geospatial data contains a *geometry* field, which can be parsed into geometric shapes and loaded into GeoPandas for geospatial data analysis

In [10]:
neighbourhoods_tables = get_package_tables("neighbourhoods")
print(f"Tables found: {len(neighbourhoods_tables)}")

Tables found: 1


In [11]:
neighbourhoods = neighbourhoods_tables[0]
neighbourhoods.head()

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,3361,25886861,25926662,49885,094,094,Wychwood (94),Wychwood (94),None,None,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4359..."
1,3362,25886820,25926663,49885,100,100,Yonge-Eglinton (100),Yonge-Eglinton (100),None,None,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4109..."
2,3363,25886834,25926664,49885,097,097,Yonge-St.Clair (97),Yonge-St.Clair (97),None,None,-79.397871,43.687859,16491537,2.222464e+06,8130.411276,"{""type"": ""Polygon"", ""coordinates"": [[[-79.3911..."
3,3364,25886593,25926665,49885,027,027,York University Heights (27),York University Heights (27),None,None,-79.488883,43.765736,16491553,2.541821e+07,25632.335242,"{""type"": ""Polygon"", ""coordinates"": [[[-79.5052..."
4,3365,25886688,25926666,49885,031,031,Yorkdale-Glen Park (31),Yorkdale-Glen Park (31),None,None,-79.457108,43.714672,16491569,1.156669e+07,13953.408098,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4396..."


Fields `['X', 'Y', 'LONGITUDE', 'LATITUDE', 'OBJECTID', 'Shape__Area', 'Shape__Length']` should be dropped. All the information in these fields is container in `geometry`, which can be updated dynamically using GeoPandas if projections change.

These fields were generated automatically by the GIS that produced the data. We plan to remove them in the future.

In [12]:
neighbourhoods = gpd.GeoDataFrame(neighbourhoods.drop(['X', 'Y', 'LONGITUDE', 'LATITUDE', 'OBJECTID', 'Shape__Area', 'Shape__Length'], axis=1))
neighbourhoods['geometry'] = neighbourhoods['geometry'].apply( lambda x: shape(json.loads(x)) )
neighbourhoods.head()

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,geometry
0,3361,25886861,25926662,49885,094,094,Wychwood (94),Wychwood (94),"POLYGON ((-79.43592 43.68015, -79.43492 43.680..."
1,3362,25886820,25926663,49885,100,100,Yonge-Eglinton (100),Yonge-Eglinton (100),"POLYGON ((-79.41096 43.70408, -79.40962 43.704..."
2,3363,25886834,25926664,49885,097,097,Yonge-St.Clair (97),Yonge-St.Clair (97),"POLYGON ((-79.39119 43.68108, -79.39141 43.680..."
3,3364,25886593,25926665,49885,027,027,York University Heights (27),York University Heights (27),"POLYGON ((-79.50529 43.75987, -79.50488 43.759..."
4,3365,25886688,25926666,49885,031,031,Yorkdale-Glen Park (31),Yorkdale-Glen Park (31),"POLYGON ((-79.43969 43.70561, -79.44011 43.705..."


#### Solid Waste Pickup Schedule

https://open.toronto.ca/dataset/solid-waste-pickup-schedule/

In [13]:
waste_tables = get_package_tables("solid-waste-pickup-schedule")
print(f"Tables found: {len(waste_tables)}")
waste = waste_tables[0]

waste.head()

Tables found: 6


,_id,Calendar,WeekStarting,GreenBin,Garbage,Recycling,YardWaste,ChristmasTree
0,1,MondayNight,2020-01-06T00:00:00,M,M,0,0,M
1,2,MondayNight,2020-01-13T00:00:00,M,0,M,0,0
2,3,MondayNight,2020-01-20T00:00:00,M,M,0,0,M
3,4,MondayNight,2020-01-27T00:00:00,M,0,M,0,0
4,5,MondayNight,2020-02-03T00:00:00,M,M,0,0,0


## Data Visualization: Fire Incidents and Neighbourhoods
**Note**: Requirement: install the jupyter-leaflet extension to view the interactive map in the notebook

In [14]:
centroids = neighbourhoods['geometry'].centroid
center = [centroids.y.mean(), centroids.x.mean()]

m = Map(
    center=center,
    basemap=basemaps.CartoDB.Positron,
)

neighbourhoods_layer = GeoData(geo_dataframe = neighbourhoods,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.5, 'weight':2, 'fillOpacity':0.3},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Neighbourhoods')

m.add_layer(neighbourhoods_layer)

In [15]:
incidents_by_year = {}

fire_icon = AwesomeIcon(
    name="fa-fire",
    marker_color="red",
    icon_color="white"
)

for idx, i in incidents.iterrows():
    year = i["TFS_Alarm_Time"].year
    if year not in incidents_by_year:
        incidents_by_year[year] = []
        
    location = [i["Latitude"], i["Longitude"]]
    marker = Marker(
        location=[i["Latitude"], i["Longitude"]],
        draggable=False,
        name=i["Incident_Number"],
        title=i["Incident_Number"],
        icon=fire_icon
    )
    incidents_by_year[year].append(marker)
    
for year in incidents_by_year:
    m.add_layer(MarkerCluster(markers=incidents_by_year[year], name=str(year)))

In [16]:
control = LayersControl(position="topright")
m.add_control(control)

In [17]:
m.layout.height="700px"
m.zoom = 11
m

Map(center=[43.708840894331885, -79.40018555801055], controls=(ZoomControl(options=['position', 'zoom_in_text'…